# LONGITUDINAL CHARACTERISTICS VECTOR

This Jupyter Notebook contains main functions to create a longitudinal characteristics vector for each ASV in the dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels import sandbox

from arch.unitroot import VarianceRatio, ZivotAndrews

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

from skbio.stats import composition

from scipy import stats
from scipy import signal
from scipy.fft import fft, fftfreq, ifft, rfft, rfftfreq

from cmath import phase
import math

from collections import Counter
import librosa

In [2]:
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1

In [3]:
wd =  './data/ready_files/'

male_df = pd.read_csv(wd + 'male_rarefied_18000_interpolated_pchip.tsv', sep = '\t', index_col = [0]).T
female_df = pd.read_csv(wd + 'female_rarefied_18000_interpolated_pchip.tsv', sep = '\t', index_col = [0]).T
donorA_df = pd.read_csv(wd + 'donorA_rarefied_18000_interpolated_pchip.tsv', sep = '\t', index_col = [0]).T
donorB_df = pd.read_csv(wd + 'donorB_rarefied_18000_interpolated_pchip.tsv', sep ='\t', index_col = [0]).T

def filter_dataset(data, treshold=150):
    
    df = data.iloc[:treshold]
    df_sum = df.sum().reset_index().sort_values(by = [0])
    keep_features = df_sum[df_sum[0] != 0]['index'].values
    data_filtered = df[keep_features]
    
    return data_filtered

#donorB_df = filter_dataset(donorB_df)

datasets = [male_df, female_df, donorA_df, donorB_df]
subjects = ['male', 'female', 'donorA', 'donorB']

In [4]:
male_df.shape, female_df.shape, donorA_df.shape, donorB_df.shape

((443, 1253), (185, 551), (365, 1524), (253, 1569))

# MEAN AND VARIANCE OVER TIME

In [5]:
MEAN_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    mean_df = dataset.mean().reset_index().rename({'index':'feature', 0:'mean'}, axis=1)
    mean_df['subject'] = subject
    MEAN_DF = pd.concat([MEAN_DF, mean_df])

In [6]:
STD_DF = pd.DataFrame()    
for dataset, subject in zip(datasets, subjects):
    
    std_df = dataset.std().reset_index().rename({'index':'feature', 0:'std'}, axis=1)
    std_df['subject'] = subject
    STD_DF = pd.concat([STD_DF, std_df])

# WHITE NOISE ANALYSIS
### test if feature is white noise based on:
    1. Ljung-Box test for autocorrelation presence
    2. Acf analysis: white noise won't exhibit autocorrelation
    3. spectrum analysis: white noise will have a flat spectrum

### 1. Ljung-Box test

In [7]:
def remove_trend(ts):
    
    lr = LinearRegression()
    X = ts.index.values.reshape(len(ts), 1)
    lr.fit(X, ts.values)
    trend = lr.predict(X)

    feature_detrended = ts.values - trend
    
    return feature_detrended

def test_for_white_noise(ts):
    
    detrended_ts = remove_trend(ts)
    
    # Ljung-Box test for white noise
    ljung_box_results = acorr_ljungbox(np.array(detrended_ts), lags=len(detrended_ts)//2)
    ljung_box_results_df = ljung_box_results.reset_index()

    return ljung_box_results_df['lb_pvalue'].mean()
    
def run_Ljung_Box_test(df, subject):
    
    white_noise_results = []
    for col in df.columns:
        white_noise_results.append({'feature':col,
                                    'ljung_box_noise': test_for_white_noise(df[col])})
    white_noise_results_df = pd.DataFrame.from_dict(white_noise_results)
    white_noise_results_df['subject'] = subject
    
    return white_noise_results_df

LjungBox_df = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    df = run_Ljung_Box_test(dataset, subject)
    LjungBox_df = pd.concat([LjungBox_df, df])

LjungBox_df = LjungBox_df.reset_index(drop=True)

In [8]:
LjungBox_df.head()

,feature,ljung_box_noise,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,2.510707e-21,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,2.430337e-67,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,9.873375e-01,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,1.135457e-01,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,1.232550e-04,male


### 2. Autocorrelation test

In [9]:
def test_noise_acf(df, subject):

    acf_noise_df = []
    for feature in df.columns:

        ts = df[feature]
        acf_vals, acf_ci, acf_qstat, acf_pvalues = sm.tsa.stattools.acf(ts, nlags=((len(ts)//2)), fft=False, alpha=0.05, qstat=True)
        centered_acf_ci = acf_ci - np.stack([acf_vals, acf_vals], axis=1)

        acf_df = pd.DataFrame(list(zip(np.abs(acf_vals), np.abs(centered_acf_ci[:, 0]))), columns = ['acf', 'ci']).iloc[1:]

        if acf_df[acf_df['acf'] > acf_df['ci']].shape[0] == 0 :
            acf_noise_df.append({'feature':feature,
                                 'acf_noise': 1})
        else: 
            acf_noise_df.append({'feature':feature,
                                 'acf_noise': 0})
    acf_white_noise_results_df = pd.DataFrame.from_dict(acf_noise_df)
    acf_white_noise_results_df['subject'] = subject
    
    return acf_white_noise_results_df

ACF_NOISE_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    corr_df = test_noise_acf(dataset, subject)
    ACF_NOISE_DF = pd.concat([ACF_NOISE_DF, corr_df])

In [10]:
ACF_NOISE_DF.head()

,feature,acf_noise,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,0,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,0,male


### 3. spectrum flatness

In [11]:
def calculate_flatness(df, subject):
    
    flatness_df = []
    for col in df.columns:

        ts = df[col]
        detrended_ts = remove_trend(ts).astype(float)
        f, t, Sxx = signal.spectrogram(ts) #get spectrogram
        flatness = librosa.feature.spectral_flatness(S= Sxx, n_fft=len(ts))
        flatness_df.append({'feature':col,
                            'flattness_score': flatness[0][0]})
    flatness_df = pd.DataFrame.from_dict(flatness_df)
    flatness_df['subject'] = subject
    
    return flatness_df

FLATNESS_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    flat_df = calculate_flatness(dataset, subject)
    FLATNESS_DF = pd.concat([FLATNESS_DF, flat_df])

In [12]:
FLATNESS_DF.head()

,feature,flattness_score,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.064479,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,0.002518,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0.743004,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0.143083,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,0.108351,male


### MERGE RESULTS FROM THREE TESTS INTO ONE DATAFRAME

In [13]:
NOISE_DF = pd.merge(ACF_NOISE_DF, LjungBox_df, on = ['feature', 'subject'], how='outer')
WHITE_NOISE_DF = pd.merge(FLATNESS_DF, NOISE_DF, on = ['feature', 'subject'], how='outer')
WHITE_NOISE_DF = WHITE_NOISE_DF[['feature', 'acf_noise', 'ljung_box_noise', 'flattness_score', 'subject']].copy()

### DEFINE WHITE NOISE

Based on our analyses we define that an ASV exhibits white noise behavior if it's flatness score is above 0.4 and if it is not autocorrelated with any of its previous lags

In [14]:
def noise_flag_df(df):
    
    if (df['flattness_score'] >= 0.4) and (df['ljung_box_noise'] > 0.05) :
        return 1
    else: return 0
    
WHITE_NOISE_DF['white_noise_binary'] = WHITE_NOISE_DF.apply(noise_flag_df, axis = 1)
WHITE_NOISE_DF.head()

,feature,acf_noise,ljung_box_noise,flattness_score,subject,white_noise_binary
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0,2.510707e-21,0.064479,male,0
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,0,2.430337e-67,0.002518,male,0
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0,9.873375e-01,0.743004,male,1
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0,1.135457e-01,0.143083,male,0
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,0,1.232550e-04,0.108351,male,0


# FEATURE LOADING

In [15]:
def analyse_pca_loadings(dataset, subject):
    
    X = composition.clr(dataset+1)

    pca = PCA(n_components=2)

    pca.fit(X)
    pca.explained_variance_ratio_

    loadings_df = pd.DataFrame(pca.components_, columns = dataset.columns, index = ['PC1', 'PC2']).T.reset_index().rename({'index':'feature'}, axis=1)
    loadings_df['PC1_loading'] = np.abs(loadings_df['PC1'])
    loadings_df['PC2_loading'] = np.abs(loadings_df['PC2'])
    loadings_df['subject'] = subject
    
    return loadings_df

PCA_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    pca_res = analyse_pca_loadings(dataset, subject)
    PCA_DF = pd.concat([PCA_DF, pca_res])

In [16]:
PCA_DF.head()

,feature,PC1,PC2,PC1_loading,PC2_loading,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,-0.007952,-0.037409,0.007952,0.037409,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,-0.001641,-0.006990,0.001641,0.006990,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,-0.004286,-0.003760,0.004286,0.003760,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,-0.005201,-0.001198,0.005201,0.001198,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,-0.008881,-0.016868,0.008881,0.016868,male


# STATIONARITY

1. KPSS test

2. ADF test

    Case 1: Both tests conclude that the series is not stationary - The series is not stationary

    Case 2: Both tests conclude that the series is stationary - The series is stationary

    Case 3: KPSS indicates stationarity and ADF indicates non-stationarity - The series is trend stationary. Trend needs to be removed to make series strict stationary. The detrended series is checked for stationarity.

    Case 4: KPSS indicates non-stationarity and ADF indicates stationarity - The series is difference stationary. Differencing is to be used to make series stationary. The differenced series is checked for stationarity.

In [17]:
def remove_trend(ts):
    
    lr = LinearRegression()
    X = ts.index.values.reshape(len(ts), 1)
    lr.fit(X, ts.values)
    trend = lr.predict(X)

    feature_detrended = ts.values - trend
    
    return feature_detrended

def test_stationarity(df, subject):
    
    results = []
    for col in df.columns:
        ts = df[col]
        try:
            detrend_ts = remove_trend(ts) #remove trend 

            result_ADF = adfuller(ts, maxlag=30) #unit root
            result_KPSS = kpss(np.log1p(ts), nlags=30, regression = 'ct') #trend stationary #Data is normally log-transformed before running the KPSS test, to turn any exponential trends into linear ones.


            results.append({'feature': col,
                            'ADF_pvalue': result_ADF[1],
                            'KPSS_pvalue':result_KPSS[1]})
        except: 
            results.append({'feature': col,
                            'ADF_pvalue': None,
                            'KPSS_pvalue':None})
            
    
    rw_results_df = pd.DataFrame.from_dict(results)
    rw_results_df['ADF_stat'] = np.where(rw_results_df.ADF_pvalue < 0.05, 0, 1) # 1 if non stationary
    rw_results_df['KPSS_stat'] = np.where(rw_results_df.KPSS_pvalue < 0.05, 1, 0) # 1 if non stationary
    rw_results_df['subject'] = subject
    
    return rw_results_df


STATIONARITY_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    df = test_stationarity(dataset, subject)
    STATIONARITY_DF = pd.concat([STATIONARITY_DF, df])
STATIONARITY_DF = STATIONARITY_DF.reset_index(drop=True)

def flag_df(df):
    
    if pd.isnull(df['ADF_pvalue']) or pd.isnull(df['KPSS_pvalue']):
        return None
    elif (df['ADF_stat'] == 1) and (df['KPSS_stat'] == 1):
        return 'non-stationary'
    elif (df['ADF_stat'] == 0) and (df['KPSS_stat'] == 0):
        return 'stationary'
    elif (df['ADF_stat'] == 1) and (df['KPSS_stat'] == 0) :
        return 'trend-stationary'
    elif (df['ADF_stat'] == 0) and (df['KPSS_stat'] == 1):
        return 'diff-stationary'
    
    
STATIONARITY_DF['stationarity'] = STATIONARITY_DF.apply(flag_df, axis = 1)

In [18]:
STATIONARITY_DF.head()

,feature,ADF_pvalue,KPSS_pvalue,ADF_stat,KPSS_stat,subject,stationarity
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,3.415240e-01,0.049024,1,1,male,non-stationary
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,7.575700e-05,0.100000,0,0,male,stationary
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0.000000e+00,0.100000,0,0,male,stationary
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,6.598690e-26,0.100000,0,0,male,stationary
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,7.333369e-04,0.100000,0,0,male,stationary


# SEASONALITY ANALYSIS

### 1. Define stationarity type to make it stationary for seasonality analysis

In [19]:
STATIONARITY_DF['stationary'] = np.where(STATIONARITY_DF['stationarity'] == 'stationary', 1, 0)
STATIONARITY_DF['non-stationary'] = np.where(STATIONARITY_DF['stationarity'] == 'non-stationary', 1, 0)
STATIONARITY_DF['trend-stationary'] = np.where(STATIONARITY_DF['stationarity'] == 'trend-stationary', 1, 0)
STATIONARITY_DF['diff-stationary'] = np.where(STATIONARITY_DF['stationarity'] == 'diff-stationary', 1, 0)


STATIONARY_TS = []
for i in range(0, 4):
    
    type1 = STATIONARITY_DF[(STATIONARITY_DF['stationarity'] == 'trend-stationary') & (STATIONARITY_DF['subject'] == subjects[i])].feature.values
    type2 = STATIONARITY_DF[(STATIONARITY_DF['stationarity'] == 'diff-stationary') & (STATIONARITY_DF['subject'] == subjects[i])].feature.values
    type3 = STATIONARITY_DF[(STATIONARITY_DF['stationarity'] == 'non-stationary') & (STATIONARITY_DF['subject'] == subjects[i])].feature.values


    data = datasets[i].copy()
    data[type1] = data[type1].apply(lambda x : remove_trend(x))
    data[type2] = data[type2].apply(lambda x : x.diff())
    data[type3] = data[type3].apply(lambda x : x.diff())
    data['subject'] = subjects[i]
    
    STATIONARY_TS.append(data)

### 2. Find how many modes is enough to explain feature

In [20]:
def explain_ts_with_fft(ts, n_modes, subject):
    
    
    ts = ts.iloc[:150]
    # Smoothin using rolling mean
    rolling_ts = ts.rolling(window=3).mean().dropna().values
    # Removing trend unig linear regression
    x = rolling_ts.reshape(len(rolling_ts), )
    
    n_modes = n_modes
    dt = 1 
    n = len(x) 
    fhat = np.fft.fft(x, n) 

    psd = fhat * np.conj(fhat)/n 
    freq = (1/(dt*n)) * np.arange(n) 
    idxs_half = np.arange(1, np.floor(n/2), dtype=np.int32)
    period = 1/freq 

    train_fft_df = pd.DataFrame(list(zip(psd[idxs_half], np.real(psd[idxs_half]), period[idxs_half], freq[idxs_half])), 
                          columns = ['pds', 'pds_real', 'period [days]', 'freq [1/day]'])
    train_fft_df = train_fft_df.sort_values(by = ['pds_real'], ascending = False)
    train_fft_df = train_fft_df[(train_fft_df['period [days]'] < len(ts)//2) & (train_fft_df['period [days]'] > 2)]

    ### Filter signal only usig dominant mode 
    threshold = train_fft_df['pds_real'].values[0:n_modes]
    psd_idxs = np.isin(psd, threshold)
    psd_clean = psd * psd_idxs   #zero out all the unnecessary powers
    fhat_clean = psd_idxs * fhat 

    signal_filtered = np.fft.ifft(fhat_clean)
    score = np.round(stats.spearmanr(signal_filtered, rolling_ts)[0], 2)
    
    return score, train_fft_df.iloc[:n_modes], 

def calculate_fft(dataset):

    df = dataset.copy()
    fft_results = []
    for col in df.columns:
        for i in range(1, 11):
            corr, res = explain_ts_with_fft(df[col], i, 'i')
            fft_results.append({'feature':col,
                                'seasonal_reconstruction_score':corr,
                                'n_modes':i,
                               'seasonality': res['period [days]'].values.tolist()})

    fft_results_df = pd.DataFrame.from_dict(fft_results)
    return fft_results_df

explained_fft_df = pd.DataFrame()
for i in range(0, 4):
    subject = STATIONARY_TS[i].iloc[:, -1].values[0]
    dataset = STATIONARY_TS[i].iloc[:, :-1]
    df = calculate_fft(dataset)
    df['subject'] = subject
    explained_fft_df = pd.concat([explained_fft_df, df])
    
    
explained_fft_df = explained_fft_df.reset_index(drop=True)

In [21]:
explained_fft_df.head()

,feature,seasonal_reconstruction_score,n_modes,seasonality,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.29,1,[9.799999999999999],male
1,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.39,2,"[9.799999999999999, 9.1875]",male
2,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.45,3,"[9.799999999999999, 9.1875, 16.333333333333332]",male
3,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.47,4,"[9.799999999999999, 9.1875, 16.333333333333332...",male
4,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.46,5,"[9.799999999999999, 9.1875, 16.333333333333332...",male


In [22]:
explained_fft_df.to_csv('./data/ts_charactericstics_tables/explained_fft_df.csv', index = False)

### 3. Define seasonal bacteria. We define that an ASV is seasonal if it's seasonal reconstruction score for  modes is at least 0.5

In [23]:
modes_saturation = explained_fft_df[explained_fft_df['n_modes'] == 6]
modes_saturation['seasonal'] = np.where(modes_saturation['seasonal_reconstruction_score'] > 0.5, 1, 0)
SEASONAL_BACTERIA_DF = modes_saturation[['feature', 'subject', 'seasonal']]
SEASONAL_BACTERIA_DF.head()

,feature,subject,seasonal
5,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male,0
15,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,male,0
25,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,male,0
35,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,male,1
45,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,male,0


### 4. Find how many modes is necesarry to get seasonal recontruction score of 0.5

In [24]:
def find_seasonality_saturation(treshold, subject):
    
    subject_df = explained_fft_df[explained_fft_df['subject'] == subject]

    results = []
    for feature in subject_df.feature.unique():
        df = subject_df[subject_df['feature'] == feature]
        corr_df = df[df['seasonal_reconstruction_score'] >= treshold]

        if corr_df.shape[0] == 0:
            results.append({
                'feature': feature,
                'max_mode': 0,
                'seasonal_reconstruction_score': 0
            })

        elif corr_df.shape[0] > 0:
            results.append({
                'feature': feature,
                'max_mode': corr_df.iloc[0].n_modes,
                'seasonal_reconstruction_score': corr_df.iloc[0]['seasonal_reconstruction_score']
            })

    results_df = pd.DataFrame.from_dict(results)
    results_df['subject'] = subject
    
    return results_df


SEASONALITY_SATURATION_DF = pd.DataFrame()
for subject in subjects:
    df = find_seasonality_saturation(0.4, subject)
    SEASONALITY_SATURATION_DF = pd.concat([SEASONALITY_SATURATION_DF, df])
    
SEASONALITY_SATURATION_DF.columns = ['feature', 'max_fft_mode', 'max_fft_mode_corr', 'subject']
SEASONALITY_SATURATION_DF.head()

,feature,max_fft_mode,max_fft_mode_corr,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,3,0.45,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,1,0.43,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,2,0.42,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,1,0.41,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,3,0.42,male


### 5. Find dominant mode
We use an adjustment here to asses whether the dominant mode is not an artifact. We only take into account seasonality that has a seasonal reconstruction score of at elast 0.4

In [25]:
ONE_MODE_DF = explained_fft_df[explained_fft_df['n_modes'] == 1]
ONE_MODE_DF['seasonality'] = [int(i[0]) for i in ONE_MODE_DF['seasonality'].values]
ONE_MODE_DF['s1_adj'] = np.where(ONE_MODE_DF['seasonal_reconstruction_score'] >= 0.4, ONE_MODE_DF['seasonality'], 0)
ONE_MODE_DF.columns = ['feature', '1st_mode_spearman_corr', 'n', '1st_mode_seasonality', 'subject', '1st_mode_adj']
ONE_MODE_DF.head()

,feature,1st_mode_spearman_corr,n,1st_mode_seasonality,subject,1st_mode_adj
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.29,1,9,male,0
10,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,0.43,1,74,male,74
20,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0.35,1,29,male,0
30,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0.41,1,29,male,29
40,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,0.20,1,13,male,0


# TREND

In [26]:
def find_trend(ts):
    
    lr = LinearRegression()
    X = ts.index.values.reshape(len(ts), 1)
    lr.fit((X), ts.values)
    trend = lr.predict(X)

    feature_detrended = ts.values - trend
    
    return lr.coef_[0]


def analyse_trend_in_bacteria(df, subject):
    
    trend_results = []
    for col in df.columns:
        trend = find_trend(df[col])
        trend_results.append({'feature':col, 
                              'trend':trend})

    trend_results_df = pd.DataFrame.from_dict(trend_results)
    trend_results_df['subject'] = subject
    
    return trend_results_df

TREND_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    df = analyse_trend_in_bacteria(dataset, subject)
    TREND_DF = pd.concat([TREND_DF, df])
    
TREND_DF = TREND_DF.reset_index(drop=True)
TREND_DF.head()

,feature,trend,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,-0.016108,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,-0.000726,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,-0.000176,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0.000143,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,-0.002487,male


#  PREVALENCE

In [27]:
def get_prevalence(df):
    
    prevalence_df = df.copy()
    prevalence_df[prevalence_df>=1] = 1
    prevalence_prc_df = prevalence_df.sum()/len(df)
    
    return prevalence_prc_df

PREVALENCE_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    prevalence_df = get_prevalence(dataset)
    prevalence_df = prevalence_df.reset_index().rename({'index':'feature', 0:'prc_occurence'}, axis=1)
    prevalence_df['subject'] = subject
    PREVALENCE_DF = pd.concat([PREVALENCE_DF, prevalence_df])
    
PREVALENCE_DF.head()

,feature,prc_occurence,subject
0,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,0.349887,male
1,TACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCG...,0.020316,male
2,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0.015801,male
3,TACGTAGGTCCCGAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAG...,0.124153,male
4,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCG...,0.221219,male


# AUTOCORRELATION

In [28]:
def analyse_autocorrelation(df, subject):
    ACF_DF = pd.DataFrame()
    for col in df.columns:
        
        ts = df[col]
        acf_vals, acf_ci, acf_qstat, acf_pvalues = sm.tsa.stattools.acf(ts, nlags=10, fft=False, alpha=0.05, qstat=True)
        acf_df = pd.DataFrame(list(zip(acf_vals, acf_pvalues)), columns = ['acf', 'pval'])
        acf_df['acf_adj'] = np.where(acf_df.pval < 0.05, acf_df['acf'], 0)
        acf_df['lag'] = np.arange(0, 10)
        acf_df['feature'] = col
        acf_df['subject'] = subject
        ACF_DF = pd.concat([ACF_DF, acf_df])
    return ACF_DF


AUTOCORR_DF = pd.DataFrame()
for dataset, subject in zip(datasets, subjects):
    
    corr_df = analyse_autocorrelation(dataset, subject)
    AUTOCORR_DF = pd.concat([AUTOCORR_DF, corr_df])
AUTOCORR_DF = AUTOCORR_DF[AUTOCORR_DF['lag'] != 0]
AUTOCORR_DF.head()

,acf,pval,acf_adj,lag,feature,subject
1,0.479820,6.720404e-35,0.479820,1,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male
2,0.349770,2.291916e-40,0.349770,2,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male
3,0.258632,5.563807e-41,0.258632,3,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male
4,0.126756,7.802735e-41,0.126756,4,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male
5,0.086832,3.137285e-40,0.086832,5,TACGGAGGGTGCGAGCGTTAATCGGAATAACTGGGCGTAAAGGGCA...,male


In [29]:
AUTOCORR_DF.to_csv('./data/ts_charactericstics_tables/autocorrelation.csv', index=False)

# MERGE FEATURES INTO A CHARACTERICTIS TABLE

In [30]:
# rename certain columns
PCA_DF = PCA_DF[['feature', 'PC1_loading', 'PC2_loading', 'subject']]
SEASONALITY_SATURATION_DF.columns = ['feature', 'n_modes', 'seasonal_reconstruction_score', 'subject']
ONE_MODE_DF.columns = ['feature', 'dominant_mode_score', 'n', 'dominant_seasonality', 'subject', 'dominant_seasonality_adj']
ONE_MODE_DF = ONE_MODE_DF[['feature', 'dominant_seasonality', 'dominant_seasonality_adj', 'dominant_mode_score', 'subject']]
PREVALENCE_DF = PREVALENCE_DF.rename({'prc_occurence':'prevalence'}, axis=1)
AUTOCORR_DF.columns = ['autocorrelation', 'pvalue', 'autocorrelation_sig', 'lag', 'feature', 'subject']
AUTOCORR_DF = AUTOCORR_DF[['feature', 'lag', 'autocorrelation', 'pvalue', 'autocorrelation_sig', 'subject']]

#set index to merge later
MEAN_DF.set_index(['feature', 'subject'],inplace=True)
STD_DF.set_index(['feature', 'subject'],inplace=True)
WHITE_NOISE_DF.set_index(['feature', 'subject'],inplace=True)
PCA_DF.set_index(['feature', 'subject'],inplace=True)
STATIONARITY_DF.set_index(['feature', 'subject'],inplace=True)
SEASONALITY_SATURATION_DF.set_index(['feature', 'subject'],inplace=True)
ONE_MODE_DF.set_index(['feature', 'subject'],inplace=True)
TREND_DF.set_index(['feature', 'subject'],inplace=True)
PREVALENCE_DF.set_index(['feature', 'subject'],inplace=True)
AUTOCORR_DF = AUTOCORR_DF.pivot(index=['feature', 'subject'], columns='lag', values='autocorrelation_sig')
AUTOCORR_DF.columns = [f'lag_{i}_corr' for i in range(1, 10)]
SEASONAL_BACTERIA_DF.set_index(['feature', 'subject'],inplace=True)

#merge
LONGITUDINAL_CHARACTERISTICS_DF = pd.concat([MEAN_DF, STD_DF, WHITE_NOISE_DF,PCA_DF, STATIONARITY_DF, \
                                             SEASONALITY_SATURATION_DF, ONE_MODE_DF, SEASONAL_BACTERIA_DF, TREND_DF, PREVALENCE_DF, \
                                             AUTOCORR_DF ], axis=1).reset_index()


#merge all non stationary variables to one
LONGITUDINAL_CHARACTERISTICS_DF['non_stationary'] = np.where((LONGITUDINAL_CHARACTERISTICS_DF['non-stationary'] == 1) | 
                               (LONGITUDINAL_CHARACTERISTICS_DF['trend-stationary'] == 1) | 
                               (LONGITUDINAL_CHARACTERISTICS_DF['diff-stationary'] == 1),
                               1, 0)

LONGITUDINAL_CHARACTERISTICS_DF = np.round(LONGITUDINAL_CHARACTERISTICS_DF, 3)
LONGITUDINAL_CHARACTERISTICS_DF.to_csv('./data/ts_charactericstics_tables/LONGITUDINAL_CHARACTERISTICS_DF.csv', index=False)

In [31]:
LONGITUDINAL_CHARACTERISTICS_DF.head()

,feature,subject,mean,std,acf_noise,ljung_box_noise,flattness_score,white_noise_binary,PC1_loading,PC2_loading,...,lag_1_corr,lag_2_corr,lag_3_corr,lag_4_corr,lag_5_corr,lag_6_corr,lag_7_corr,lag_8_corr,lag_9_corr,non_stationary
0,AACATAGGGGGCAAGCGTTATCCGGAATCACTGGGCGTAAAGGGCG...,male,0.002,0.048,1,0.999,0.965,1,0.004,0.003,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0
1,AACATAGGGGGCAAGCGTTGTCCGGAATCACTGGGCGTAAAGGGCG...,male,0.002,0.048,1,1.000,0.928,1,0.004,0.003,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0
2,AACATAGGGGGCAAGCGTTGTCCGGAATCACTGGGCGTAAAGGGCG...,donorB,0.012,0.140,0,0.856,0.461,1,0.003,0.005,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0
3,AACATAGGGGGCAAGCGTTGTCCGGAATCACTGGGCGTAAAGGGCG...,female,0.027,0.264,0,0.859,0.317,0,0.007,0.003,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0
4,AACATAGGGGGCAAGCGTTGTCCGGAATCACTGGGCGTAAAGGGCG...,male,2.384,6.752,0,0.000,0.093,0,0.071,0.000,...,0.307,0.169,0.245,0.046,-0.03,-0.041,-0.063,0.014,-0.041,0
